In [1]:
import sys
sys.path.append('..')
import geedatasets

In [2]:
import ee
ee.Initialize()

In [3]:
import ipygee as ui

In [4]:
Map = ui.Map()
Map.show()

Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…

## Rescale Landsat 8 SR to Landsat 8 TOA

In [5]:
l8sr = geedatasets.fromShortName('L8SR')
l8toa = geedatasets.fromShortName('L8TOA')

In [6]:
p = ee.Geometry.Point([-72, -43])
Map.centerObject(p, 13)

In [11]:
il8sr = l8sr.collection(p, ('2019-01-10', '2019-01-20')).first()
il8toa = l8toa.collection(p, ('2019-01-10', '2019-01-20')).first()

In [17]:
vis_sr = l8sr.visualization('NSR')
vis_toa = l8toa.visualization('NSR')

In [14]:
Map.addLayer(il8sr, vis_sr, 'Original L8 SR image')
Map.addLayer(il8toa, vis_toa, 'Original L8 TOA image')

In [15]:
# RESCALE
il8sr_to_toa = l8sr.rescale(il8sr, l8toa)
il8toa_to_sr = l8toa.rescale(il8toa, l8sr)

In [18]:
Map.addLayer(il8sr_to_toa, dict(bands=vis_sr['bands'], min=vis_toa['min'], max=vis_toa['max']), 'L8 SR image rescaled to TOA')
Map.addLayer(il8toa_to_sr, dict(bands=vis_toa['bands'], min=vis_sr['min'], max=vis_sr['max']), 'L8 TOA image rescaled to SR')

In [20]:
# VALUES
def get_values(i):
    i = i.select(['B2', 'B3', 'B4'])
    values = i.reduceRegion(geometry=p, reducer=ee.Reducer.first(), scale=10)
    return values.getInfo()

print('Original L8 SR', get_values(il8sr))
print('L8 SR rescaled to TOA', get_values(il8sr_to_toa))
print('Original L8 TOA', get_values(il8toa))
print('L8 TOA rescaled to SR', get_values(il8toa_to_sr))

Original L8 SR {'B2': 2846, 'B3': 2448, 'B4': 2313}
L8 SR rescaled to TOA {'B2': 0.28459998965263367, 'B3': 0.24480000138282776, 'B4': 0.2312999963760376}
Original L8 TOA {'B2': 0.30524054169654846, 'B3': 0.2429025024175644, 'B4': 0.22954615950584412}
L8 TOA rescaled to SR {'B2': 3052, 'B3': 2429, 'B4': 2295}
